In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
from scipy.constants import mu_0
import discretize
import time

from simpeg import maps
from simpeg.electromagnetics import time_domain as tdem

import casing_utils

In [ ]:
sigma_back = 1e-1
sigma_air = 1e-4

target_r = 1
target_l = 50

sigma_target = 1e5
mur_target = 10

In [ ]:
rx_x = np.arange(1, 101)[1::2]
rx_x

In [ ]:
src_z = np.r_[-70, -60]

In [ ]:
n_cells = [8, 4, 2, 1]

In [ ]:
meshes = {}

pf = 1.3

csz = 1
npadz = 25
n_core_z = int(np.max(np.abs(src_z))/csz)
hz = discretize.utils.unpack_widths([(csz, npadz, -pf), (csz, n_core_z), (csz, npadz, pf)])

npadx = npadz

for n in n_cells: 
    hx = casing_utils.pad_for_casing_and_data(
        target_r, 
        csx1=target_r/n,
        csx2=csz, 
        pfx1=pf, 
        pfx2=pf, 
        domain_x=np.max(rx_x), 
        npadx=npadx
    )
    meshes[n] = discretize.CylindricalMesh(
        [hx, [np.pi*2], hz], origin=np.r_[0, 0, -np.sum(hz[:npadz+n_core_z])]
    )

In [ ]:
meshes[8].plot_grid()

In [ ]:
models = {}

for n in n_cells: 
    mesh = meshes[n]
    
    sigma = np.ones(mesh.n_cells) * sigma_air
    sigma[mesh.cell_centers[:, 2] < 0] = sigma_back

    inds_target = (
        (mesh.cell_centers[:, 0] < target_r) &
        (mesh.cell_centers[:, 2] < 0) & 
        (mesh.cell_centers[:, 2] > -target_l)
    )
    sigma[inds_target] = sigma_target

    mur = np.ones(mesh.n_cells) 
    mur[inds_target] = mur_target

    models[n] = {
        "sigma":sigma,
        "mur": mur
    }

In [ ]:
n_plot = n_cells[0]

fig, ax = plt.subplots(1, 2)

mesh = meshes[n_plot]
model = models[n_plot]

plt.colorbar(mesh.plot_image(
    model["sigma"],
    pcolor_opts={"norm":LogNorm()},
    mirror=True,
    ax=ax[0],
    
)[0], ax=ax[0])

plt.colorbar(mesh.plot_image(
    model["mur"],
    pcolor_opts={"norm":Normalize()},
    mirror=True,
    ax=ax[1]
)[0], ax=ax[1])

for a in ax: 
    a.set_xlim(np.r_[-1, 1]*5),
    a.set_ylim(np.r_[-100, 20])

plt.tight_layout()

In [ ]:
def get_survey(rx_times = np.logspace(1e-5, 1e-2, 30)): 
    rx = tdem.receivers.PointElectricField(
        locations=discretize.utils.ndgrid(rx_x, np.r_[0], np.r_[0]),
        orientation="x",
        times=rx_times, 
    )
    
    src = tdem.sources.LineCurrent(
        receiver_list = [rx],
        location = np.array([
            [0, 0, src_z.min()], 
            [0, 0, src_z.max()], 
        ])
    )
    
    survey = tdem.Survey([src])

In [ ]:
time_steps = [
    (1e-6, 20),
    (3e-6, 20),
    (1e-5, 20), (3e-5, 20), (1e-4, 20), (3e-4, 20)
]

In [ ]:
simulations = {}

for n in n_cells: 
    mesh = meshes[n]
    survey = get_survey()
    simulations[n] = tdem.Simulation3DMagneticField(
        mesh=mesh,
        survey=survey, 
        time_steps=time_steps, 
        sigmaMap=maps.IdentityMap(mesh), 
        solver=Solver
    )

In [ ]:
simulations[n].times

In [ ]:
data_conductive = {}
data_permeable = {}
fields_conductive = {}
fields_permeable = {}

for n, sim in simulations.items():
    print(f"Starting {n} conductive ...")
    t0 = time.time()
    fields_conductive[n] = sim.fields(models[n]["sigma"])
    dpred_conductive[n] = sim.dpred(models[n]["sigma"], f=fields_conductive[n])
    print(f" ... done. Time: {time.time()-t0:1.1e}s")

    print(f"Starting {n} conductive, permeable ...")
    t0 = time.time()
    sim.mu = mu_0 * models[n]["mur"]
    fields_permeable[n] = sim.fields(models[n]["sigma"])
    dpred_permeable[n] = sim.dpred(models[n]["sigma"], f=fields_permeable[n])
    print(f" ... done. Time: {time.time()-t0:1.1e}s")